In [1]:
from platform import python_version

print(python_version())

3.12.3


In [2]:
pip install -r requirements.txt

  Cloning https://github.com/qiskit-community/Quantum-Challenge-Grader.git to /tmp/pip-req-build-_v0o264z
  Running command git clone --filter=blob:none --quiet https://github.com/qiskit-community/Quantum-Challenge-Grader.git /tmp/pip-req-build-_v0o264z
  Resolved https://github.com/qiskit-community/Quantum-Challenge-Grader.git to commit 897f4448b14d18b4d0a430f57f11c2dcb6200c20
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

import numpy as np
from numpy import pi
import matplotlib.pyplot as plt


# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.primitives import BitArray
from qiskit.visualization import *

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.circuit import Gate
from qiskit.circuit.library import XGate

from qiskit.providers.backend import BackendV2 as Backend
from qiskit.transpiler import CouplingMap, InstructionDurations
from qiskit.transpiler.passmanager import PassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService, Options
from qiskit_ibm_runtime import Session, Batch, SamplerV2 as Sampler

from qiskit_ibm_runtime.transpiler.passes.scheduling import (
    DynamicCircuitInstructionDurations,
    ALAPScheduleAnalysis,
    PadDynamicalDecoupling,
)

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Save your API key to track your progress and have access to the quantum computers

your_api_key = "lam3AN0GvvpVGwJQk6RMDDaWKNdoPvJqo2LTooXEz0ml"
your_crn = "crn:v1:bluemix:public:quantum-computing:us-east:a/84889df94a104432b27b1d04dd4411f7:27251608-ce74-48a1-be7d-54412f5a9a59::"

QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token=your_api_key,
    instance=your_crn,
    name="qgss-2025",
    set_as_default=True,
    overwrite=True
)

In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService
from typing import Any, List, Dict, Union, Optional, Callable, Tuple
from qiskit.circuit import IfElseOp

service = QiskitRuntimeService()


In [7]:
def coupling_map_from_qubit_line(coupling_map:List[List[int]],
                                 qubit_line: List[List[int]]) -> List[List[int]]:
    """
     Modify the full coupling map to force linearity in the qubit layout
    """
    new_coupling_map = []
    line_edge_list = []
    for i in range(len(qubit_line)-1):
        line_edge_list.append([qubit_line[i],qubit_line[i+1]])

    for edge in coupling_map:
        u,v = edge
        edge_rev = [v,u]
        if (edge in line_edge_list) or (edge_rev in line_edge_list):
            new_coupling_map.append(edge)
    return new_coupling_map

In [8]:
# Set which quantum computer to use
backend = service.least_busy(operational=True)

In [10]:
backend.target.add_instruction(IfElseOp, name="if_else_op") 
MACHINE_NAME = backend.name
print(f"Using backend: {MACHINE_NAME}")

Using backend: ibm_sherbrooke


In [ ]:
#update qubit line according to your backend connectivity, here I used ibm_torino
qubit_lines = {backend.name:[0,1,2,3,4,16,23, 24, 25, 35, 44]}
# Set qubit line and coupling map
QUBIT_LINE = qubit_lines[MACHINE_NAME]
COUPLING_MAP_FULL = [list(edge) for edge in list(QiskitRuntimeService().backend(MACHINE_NAME).coupling_map)]
COUPLING_MAP_1D = coupling_map_from_qubit_line(COUPLING_MAP_FULL, QUBIT_LINE)
MAX_POSSIBLE_QUBITS_BTW_CNOT = len(QUBIT_LINE) - 2

In [23]:
print(f"Machine is set to: {MACHINE_NAME}")
print(f"Maximum number of qubits between CNOT for {MACHINE_NAME} is {MAX_POSSIBLE_QUBITS_BTW_CNOT} with the given qubit line.")

Machine is set to: ibm_sherbrooke
Maximum number of qubits between CNOT for ibm_sherbrooke is 9 with the given qubit line.


In [24]:
OPTIMIZATION_LEVEL = 3
# Set to True to use dynamical decoupling
SHOTS = 1024
sampler = Sampler(mode=backend)
sampler.options.experimental = {"execution_path" : "gen3-experimental"}